In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-dataset/loan_approval.csv


<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    background-color: #023e8a;
    color: white;
    border-radius: 10px;
    border: 2px solid #023e8a;
    font-family: 'Montserrat', sans-serif;
    font-weight: 600;
    font-size: 16px;
    text-align: center;
    padding: 12px 16px;
">
1. Importando módulos
</div>

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

<div style="
    display: flex;
    align-items: flex-start;
    background-color:#ade8f4;
    color: #023e8a;
    padding:15px 20px;
    border-left:5px solid #023e8a;
    border-radius:10px;
    max-width:100%;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
    font-family: 'Montserrat', sans-serif;
">
<ul style="margin: 0; padding-left: 20px;">
    <li><b>Pandas</b> para trabalhar com DataFrames</li>
    <li><b>train_test_split</b> serve para dividir os dados em conjuntos de treino e teste.</li>
    <li><b>StandardScaler</b> é usado para padronizar os dados, deixando as features na mesma escala.</li>
    <li><b>LogisticRegression</b> é o modelo de regressão logística, útil para problemas de classificação binária.</li>
    <li>Ignoramos avisos (<b>warnings</b>) para manter a saída do notebook mais limpa e legível.</li>
</ul>
</div>

<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    background-color: #023e8a;
    color: white;
    border-radius: 10px;
    border: 2px solid #023e8a;
    font-family: 'Montserrat', sans-serif;
    font-weight: 600;
    font-size: 16px;
    text-align: center;
    padding: 12px 16px;
">
2. Visualizando os dados
</div>

In [3]:
df = pd.read_csv("/kaggle/input/loan-approval-dataset/loan_approval.csv")
df.head(5)

,name,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,Allison Hill,East Jill,113810,389,39698,27,50.0,False
1,Brandon Hall,New Jamesside,44592,729,15446,28,55.0,False
2,Rhonda Smith,Lake Roberto,33278,584,11189,13,45.0,False
3,Gabrielle Davis,West Melanieview,127196,344,48823,29,50.0,False
4,Valerie Gray,Mariastad,66048,496,47174,4,25.0,False


<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    background-color: #023e8a;
    color: white;
    border-radius: 10px;
    border: 2px solid #023e8a;
    font-family: 'Montserrat', sans-serif;
    font-weight: 600;
    font-size: 16px;
    text-align: center;
    padding: 12px 16px;
">
2. Informações sobre os dados
</div>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            2000 non-null   object 
 1   city            2000 non-null   object 
 2   income          2000 non-null   int64  
 3   credit_score    2000 non-null   int64  
 4   loan_amount     2000 non-null   int64  
 5   years_employed  2000 non-null   int64  
 6   points          2000 non-null   float64
 7   loan_approved   2000 non-null   bool   
dtypes: bool(1), float64(1), int64(4), object(2)
memory usage: 111.5+ KB


<div style="
    display: flex;
    align-items: flex-start;
    background-color:#ade8f4;
    color: #023e8a;
    padding:15px 20px;
    border-left:5px solid #023e8a;
    border-radius:10px;
    max-width:100%;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
    font-family: 'Montserrat', sans-serif;
">
<ul style="margin: 0; padding-left: 20px;">
    <li>O dataset possui 2000 registros e 8 colunas</li>
    <li>As colunas incluem tanto <b>variáveis numéricas</b> (income, credit_score, loan_amount, years_employed, points) quanto <b>categóricas</b> (name, city) e uma variável booleana alvo (loan_approved).</li>
    <li>O tipo de dado bool indica que o problema é de <b>classificação binária</b> (empréstimo aprovado ou não).</li>
    <li>As variáveis numéricas têm tipos adequados (int64 e float64), prontos para normalização e modelagem.</li>
    <li>As colunas name e city provavelmente não têm impacto direto no modelo preditivo e podem ser removidas ou transformadas se contiverem informação útil (como localização).</li>
</ul>
</div>


<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    background-color: #023e8a;
    color: white;
    border-radius: 10px;
    border: 2px solid #023e8a;
    font-family: 'Montserrat', sans-serif;
    font-weight: 600;
    font-size: 16px;
    text-align: center;
    padding: 12px 16px;
">
3. Limpeza e tratamento
</div>

In [5]:
df.loan_approved = df.loan_approved.astype(int)
df.drop(['name', 'city', 'points'], inplace=True, axis=1)

<div style="
    display: flex;
    align-items: flex-start;
    background-color:#ade8f4;
    color: #023e8a;
    padding:15px 20px;
    border-left:5px solid #023e8a;
    border-radius:10px;
    max-width:100%;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
    font-family: 'Montserrat', sans-serif;
">
<ul style="margin: 0; padding-left: 20px;">
  <li>Converte a coluna <code>loan_approved</code> de booleana para inteiro (0/1) para uso em modelagem e métricas.</li>
  <li>Remove colunas irrelevantes ou redundantes (<code>name</code>, <code>city</code> e <code>points</code>) para simplificar o dataset.</li>
</ul>
</div>

In [6]:
# verificar nulos
df.isnull().sum()

income            0
credit_score      0
loan_amount       0
years_employed    0
loan_approved     0
dtype: int64

In [7]:
# Verificando valores duplicados
print()
print(f"Quantidade de linhas duplicadas: ", df.duplicated().sum())


Quantidade de linhas duplicadas:  0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   income          2000 non-null   int64
 1   credit_score    2000 non-null   int64
 2   loan_amount     2000 non-null   int64
 3   years_employed  2000 non-null   int64
 4   loan_approved   2000 non-null   int64
dtypes: int64(5)
memory usage: 78.3 KB


<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    background-color: #023e8a;
    color: white;
    border-radius: 10px;
    border: 2px solid #023e8a;
    font-family: 'Montserrat', sans-serif;
    font-weight: 600;
    font-size: 16px;
    text-align: center;
    padding: 12px 16px;
">
4. Iniciando a Etapa de Machine Learning
</div>

In [9]:
X = df.drop(['loan_approved'], axis=1)
y = df.loan_approved

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)

In [11]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
model = LogisticRegression()

model.fit(X_train_scaled, y_train)

LogisticRegression()

In [13]:
y_pred = model.predict(X_test_scaled)

<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    background-color: #023e8a;
    color: white;
    border-radius: 10px;
    border: 2px solid #023e8a;
    font-family: 'Montserrat', sans-serif;
    font-weight: 600;
    font-size: 16px;
    text-align: center;
    padding: 12px 16px;
    margin-bottom: 12px;
">
5. Avaliando o Desempenho do Modelo
</div>
<div style="
    background-color:#ade8f4; 
    color:#023e8a; 
    padding:15px 20px; 
    border-left:5px solid #023e8a; 
    border-radius:10px; 
    max-width:100%; 
    box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
    font-family:'Montserrat', sans-serif;
    line-height:1.6;
">
<ul style="margin:0; padding-left:20px;">
  <li><b>Acurácia (0.90):</b> o modelo acerta cerca de <b>90%</b> das previsões, indicando excelente capacidade de generalização.</li>
  <li><b>Matriz de Confusão:</b>
    <ul>
      <li><b>301</b> verdadeiros negativos (empréstimos não aprovados corretamente identificados)</li>
      <li><b>32</b> falsos positivos (aprovados incorretamente — risco de inadimplência)</li>
      <li><b>31</b> falsos negativos (negados indevidamente — perda de oportunidade)</li>
      <li><b>236</b> verdadeiros positivos (aprovações corretas)</li>
    </ul>
  </li>
  <li><b>Precisão para classe 0 (0.91):</b> quando o modelo prevê "não aprovado", ele acerta 91% das vezes.</li>
  <li><b>Precisão para classe 1 (0.88):</b> quando o modelo prevê "aprovado", ele acerta 88% das vezes.</li>
  <li><b>Revocação para classe 0 (0.90):</b> o modelo identifica corretamente 90% das pessoas que deveriam ser rejeitadas.</li>
  <li><b>Revocação para classe 1 (0.88):</b> o modelo identifica corretamente 88% das pessoas que deveriam ser aprovadas.</li>
  <li><b>F1-score (0.88-0.91):</b> excelente equilíbrio entre precisão e revocação em ambas as classes — desempenho consistente e balanceado.</li>
  <li><b>Análise Geral:</b> o modelo demonstra <b>alto desempenho balanceado</b>, com métricas similares para ambas as classes (aprovado e não aprovado), indicando que não há viés significativo e que é adequado para uso em produção.</li>
</ul>
</div>

In [14]:
# Acurácia
print("Acurácia:", accuracy_score(y_test, y_pred))

# Matriz de confusão
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))

# Relatório completo (precision, recall, F1-score)
print(classification_report(y_test, y_pred))

Acurácia: 0.895
Matriz de Confusão:
 [[301  32]
 [ 31 236]]
              precision    recall  f1-score   support

           0       0.91      0.90      0.91       333
           1       0.88      0.88      0.88       267

    accuracy                           0.90       600
   macro avg       0.89      0.89      0.89       600
weighted avg       0.90      0.90      0.90       600

